In [ ]:
// Import des librairies
import org.apache.spark.sql.SparkSession
import org.apache.spark._
import org.apache.spark.rdd._
import org.apache.spark.SparkContext._
import org.apache.spark.mllib.feature.HashingTF
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import scala.util.{Success, Try}
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.sql.functions



//LECTURE DU FICHIER D'ENTREE DES COMMENTAIRES AVEC DELIMITEUR #
val commentaires_bruts = spark.read.option("delimiter", "#")
                                    .option("multiline",true)
                                    .csv("Data/commentaires_evalues_V2.txt")

val stopwords = spark.sparkContext.textFile("French_stop_words.txt").collect()
val remover = new StopWordsRemover()
      .setStopWords(stopwords)
      .setInputCol("_c1")
      .setOutputCol("removed")

val toArray = udf[Array[String], String]( _.split("[ '-.,;:?!\\(\\)]"))
val commentaires_toarray = commentaires_bruts.withColumn("_c1", toArray(lower(commentaires_bruts("_c1"))))
val commentaires_withoutSW = remover.transform(commentaires_toarray)  

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES LUS
commentaires_withoutSW.createOrReplaceTempView("commentaires_withoutSW")

//CREATION DU NOM DES COLONNES POUR LA TABLE COMMENTAIRES
val commentaires = spark.sql("select _c0 as Id_comment, concat_ws(' ',_c1) as commentaire,concat_ws(' ',removed) as commentaire_SW, _c2 as qualite from commentaires_withoutSW")

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES FINALISES
commentaires.createOrReplaceTempView("commentaires")


In [43]:
commentaires.take(5)

1,hôte extrêmement attentif et très attaché à remettre en ordre les petits soucis mineurs qui ont pu apparaître studio sympa et très bien placéa recommander,hôte extrêmement attentif très attaché remettre ordre petits soucis mineurs apparaître studio sympa très bien placéa recommander,positif
2,aucun problème durant le séjour vincent fût toujours à l heure ainsi que disponible pour répondre à mes questions le logement était parfaitement fonctionnel et très cozy situé dans le 6e ce qui ajoute bien sûr un cadre agréable a recommander,problème durant séjour vincent fût toujours heure ainsi disponible répondre questions logement parfaitement fonctionnel très cozy situé 6e ajoute bien sûr cadre agréable recommander,positif
3,un accueil chaleureux de kathy dans cet appartement agréable et très bien placé dans paris et une organisation très professionnelle et attentive de la part de cobblestone une adresse à recommander,accueil chaleureux kathy cet appartement agréable très bien placé paris organisation très professionnelle attentive part cobblestone adresse recommander,positif
4,j ai été accueilli en debut d après midi au studio par la sympathique katia qui m a fourni tous les renseignements nécessaires à mon court sejour je l ai par ailleurs revue une fois où elle est venue m apporter les billets pour la croisière sur la seine que je ne parvenais pas à imprimer le studio est totalement conforme aux photos et à la description qui en est faite situé au 3èm étage avec ascenseur d une résidence sécurisée donnant sur une petite place il est dans un quartier très vivant et agréable à deux pas du marais où tout semble à portée de main de la boulangerie mitoyenne aux bars et restaurants voisins en passant par la pharmacie d en face 3 stations de métro très proches règlent facilement la question du déplacement petit mais fonctionnel très lumineux et bien décoré on y revient volontiers après les excursions de la journée le seul petit bémol sera pour les voisins de l immeuble d en face qui auraient dû en mettre plusieurs à la clef lors d une fête assez bruyante malgré ce qui peut arriver partout ailleurs dans une grande ville je le recommande vivement et serais prêt à y revenir volontiers,ai accueilli debut d après midi studio sympathique katia fourni renseignements nécessaires court sejour ai ailleurs revue fois venue apporter billets croisière seine ne parvenais pas imprimer studio totalement conforme photos description faite situé 3èm étage ascenseur d résidence sécurisée donnant petite place quartier très vivant agréable deux pas marais semble portée main boulangerie mitoyenne bars restaurants voisins passant pharmacie d face 3 stations métro très proches règlent facilement question déplacement petit fonctionnel très lumineux bien décoré revient volontiers après excursions journée seul petit bémol voisins immeuble d face auraient dû mettre plusieurs clef lors d fête assez bruyante malgré arriver partout ailleurs grande ville recommande vivement serais prêt revenir volontiers,positif
5,merci à l’hôte pour sa ponctualité à nous accueillir et les très bonnes indications pour s y rendre la localisation est vraiment excellente très centrale endroit très tranquille attention il faut bien considérer que la photo du studio est prise avec un grand angle les dimensions réelles sont surprenantes l appart est plutôt sombre les fenêtres donnant sur cour intérieure exiguë rapport qualité/prix intéressant,merci l’hôte ponctualité accueillir très bonnes indications s rendre localisation vraiment excellente très centrale endroit très tranquille attention faut bien considérer photo studio prise grand angle dimensions réelles sont surprenantes appart plutôt sombre fenêtres donnant cour intérieure exiguë rapport qualité/prix intéressant,positif


In [ ]:
val commentaires_negatifs = spark.sql("select * from commentaires where qualite like '%négatif%'") 
val commentaires_positifs = spark.sql("select * from commentaires where qualite like '%positif%'")
val commentaires_positifs_nbre = commentaires_positifs.count()
val commentaires_negatifs_nbre = commentaires_negatifs.count()


//DEFINITION DE LA TAILLE DE L'ECHANTILLON D'APPRENTISSAGE
val taille_echantillon = Math.min(commentaires_negatifs_nbre, commentaires_positifs_nbre).toInt

//DEFINITION DE L'ECHANTILLON D'APPRENTISSAGE AVEC AUTANT DE COMMENTAIRES POSITIFS QUE NEGATIFS
var echantillon = commentaires_positifs.limit(taille_echantillon).unionAll(commentaires_negatifs.limit(taille_echantillon))
echantillon.createOrReplaceTempView("echantillon")
val echantillon_msg = spark.sql("select concat( qualite, ' ', commentaire_SW) as msg from echantillon")
val echantillonRDD = echantillon_msg.rdd
val hashingTF = new HashingTF(20000)


In [46]:
println("Commentaires                  = "+commentaires.count())
println("   dont commentaires négatifs = "+commentaires_negatifs.count())
println("   dont commentaires positifs = "+commentaires_positifs.count())
println("Taille echantillon = "+echantillonRDD.count())


Commentaires                  = 205368
   dont commentaires négatifs = 1322
   dont commentaires positifs = 187489
Taille echantillon = 2644


In [ ]:
val echantillon_reformat = echantillonRDD.map(
  row =>{
    Try{
      val msg = row.toString.toLowerCase()
      var isHappy:Int = 0
      if(msg.contains("positif ")){
        isHappy = 1
      }else if(msg.contains("negatif ")){
        isHappy = 0
      }
      var msgSanitized = msg.replaceAll("positif ", "")
      msgSanitized = msgSanitized.replaceAll("negatif ","")
      (isHappy, msgSanitized.split(" ").toSeq)
    }
  }
)

var echantillon_ok = echantillon_reformat.filter((_.isSuccess)).map(_.get)
val echantillon_transfo = echantillon_ok.map(
  t => (t._1, hashingTF.transform(t._2)))
  .map(x => new LabeledPoint((x._1).toDouble, x._2))

var sample = echantillon_ok.take(1000).map(
  t => (t._1, hashingTF.transform(t._2), t._2))
  .map(x => (new LabeledPoint((x._1).toDouble, x._2), x._3))

val splits = echantillon_transfo.randomSplit(Array(0.7, 0.3))
val (trainingData, validationData) = (splits(0), splits(1))



In [48]:
//CONSTRUCTION DU MODELE A PARTIR DU JEU D'APPRENTISSAGE

val numIterations = 100
val model = SVMWithSGD.train(trainingData, numIterations)

//model.clearThreshold()

// SAUVEGARDE DU MODELE
model.save(sc, "modele/SVMClassification")





numIterations = 100
model = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 20000, numClasses = 2, threshold = 0.0


org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 20000, numClasses = 2, threshold = 0.0

In [ ]:
// Prédiction sur le jeu de validation
val predictionAndLabels = validationData.map { case LabeledPoint(label, features) =>
  val prediction = model.predict(features)
  (prediction, label)
}

In [ ]:
// Calcul des métriques
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val metrics2 = new BinaryClassificationMetrics(predictionAndLabels)

// AUPRC
val auPRC = 100 * (metrics2.areaUnderPR() - metrics2.areaUnderPR() % 0.0001) 

// AUROC
val auROC = 100 * (metrics2.areaUnderROC() - metrics2.areaUnderROC() % 0.0001) 
val Sensitivity = predictionAndLabels.filter(x => x._1 == x._2 && x._1 == 1.0).count().toDouble / predictionAndLabels.filter(x => x._2 == 1.0).count().toDouble
val Specificity = predictionAndLabels.filter(x => x._1 == x._2 && x._1 == 1.0).count().toDouble / predictionAndLabels.filter(x => x._2 == 1.0).count().toDouble
val Accuracy = predictionAndLabels.filter(x => x._1 == x._2).count().toDouble / predictionAndLabels.count().toDouble
 


In [51]:
//Affichage des métriques

println("Modèle SVM")

println(s"Area under precision-recall curve = $auPRC %")
println(s"Area under ROC = $auROC %")
println(s"Sensitivity = ${100*(Sensitivity-Sensitivity  % 0.0001)} %")
println(s"Specificity = ${100*(Specificity-Specificity % 0.0001)} %")
println(s"Accuracy = ${100*(Accuracy -Accuracy % 0.0001)} %")

Modèle SVM
Area under precision-recall curve = 98.84 %
Area under ROC = 98.15 %
Sensitivity = 97.31 %
Specificity = 97.31 %
Accuracy = 98.14 %
